In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from scipy.special import erf
from  plotly.graph_objects import FigureWidget
from ipywidgets import FloatRangeSlider, Layout, HBox, VBox
fname = '09_2DWrinkAu_640kx_770mm_70um_0p64mrad_ss8_50x_50y_100z_216step_x256_y256.raw'
fnum = int(fname[0:2])
yrows = 130
xcols = 128
cep_max = 1e6
bright_disk_radius = 5
erf_sharpness = 5

hann2d = np.hanning(xcols)
hann2d = np.outer(hann2d, hann2d)

kx = np.arange(-xcols, xcols, 2)/2
kx,ky = np.meshgrid(kx, kx)

dist = np.hypot(kx, ky)

haadf_mask = np.array(dist >= 30, np.int)

x = np.arange(-xcols/2,xcols/2,1)
x,y = np.meshgrid(x,x)
dist = np.hypot(x,y)
bdisk_filter = erf((dist-bright_disk_radius)*erf_sharpness)/2 - erf((dist+bright_disk_radius)*erf_sharpness)/2 + 1

hann_filter = np.hanning(xcols)
hann_filter = np.outer(hann_filter, hann_filter)

#%%
#Pulls a yrows x xcols diffraction pattern at [first_index, second_index] from filename
#Will crop to xcols x xcols if crop == True
def dp_slice(filename, first_index = 0, second_index = 0, yrows = yrows, 
             xcols = xcols, crop = True, dtype = np.float32, min_count_to_zero = 20, min_offset = 1e-6):
    
    dsize = np.dtype(dtype).itemsize
    
    num_pixels = int((os.stat(filename).st_size/yrows/xcols/dsize)**0.5)   
    offset = int((num_pixels*first_index + second_index)*yrows*xcols*dsize)    
    dp_slice = np.memmap(filename, dtype = dtype, mode = 'r', shape = (yrows, xcols), 
                         order = 'C', offset = offset)
    dp_slice = np.array(dp_slice)    
    if crop:
        dp_slice = dp_slice[:xcols, :xcols]
    
    #Counts under min_count set to min_offset to be very close to zero but not exactly zero to avoid errors
    #with taking logarithms
    dp_slice[dp_slice <= min_count_to_zero] = min_offset
    
    return dp_slice

#Transforms either dpslice or full 4D dp to cepstrum
def dp_to_cep(dp, window = hann2d):
#    cep = dp*window
#    cep[cep==0] = 0.0001
    cep = np.log10(dp)
    cep = np.fft.fft2(cep)
    cep = np.fft.fftshift(cep, (-1, -2))
    cep = np.abs(cep)**2.0 
    return cep


#Creates image from filename dp based on mask
def generate_image(filename, mask, yrows = yrows, xcols = xcols, dtype = np.float32):
    dsize = np.dtype(dtype).itemsize 
    num_pixels = int((os.stat(filename).st_size/yrows/xcols/dsize)**0.5)
    haadf = np.zeros((num_pixels, num_pixels))
    for i in range(num_pixels):
        for j in range(num_pixels):
            haadf[i,j] = np.sum(dp_slice(filename, i, j)*haadf_mask)
    return haadf

In [ ]:
im = generate_image(fname, haadf_mask)
im_traces = {
    #'type': 'heatmapgl',
    'type': 'heatmap',
    'z':im,
    'colorscale': 'Viridis',
    'showscale' : False,
    'hoverinfo' : 'none'
}
im_layout = {
  'margin': {
    't': 10,
    'r': 10,
    'b': 10,
    'l': 10
  },
  'xaxis' : {
    'showgrid':False,
    'showline':False,
    'zeroline':False,
    'autorange':'reversed',
    'showticklabels':False,
  },
  'yaxis' : {
    'showgrid':False,
    'showline':False,
    'zeroline':False,
    'showticklabels':False,
  },   
  'shapes' : [{
      'type': 'line',
      'xref': 'x',
      'yref': 'paper',
      'x0' : 0,
      'y0' : 0,
      'x1' : 0,
      'y1' : 1,
      'line' : {
          'color': 'rgb(0, 0, 0)',
          'width': 2,
      }
  }, 
  {
      'type': 'line',
      'xref': 'paper',
      'yref': 'y',
      'x0' : 0,
      'y0' : 0,
      'x1' : 1,
      'y1' : 0,
      'line' : {
          'color': 'rgb(0, 0, 0)',
          'width': 2,
      }
  }]
}
im_fig = FigureWidget(data=im_traces, layout=im_layout)

In [ ]:
dslice = dp_slice(fname, 0, 0)
dslice_traces = {
    'type': 'heatmapgl',
    #'type': 'heatmap',
    'z':dslice,
    'colorscale': 'Viridis',
    'showscale' : True,
    'hoverinfo' : 'none'
}
dslice_layout = {
  'margin': {
    't': 10,
    'r': 10,
    'b': 10,
    'l': 10
  },
  'xaxis' : {
    'showgrid':False,
    'showline':False,
    'zeroline':False,
    #'autorange':'reversed',
    'showticklabels':False,
  },
  'yaxis' : {
    'showgrid':False,
    'showline':False,
    'zeroline':False,
    'showticklabels':False,
  },   
}
dslice_fig = FigureWidget(data=dslice_traces, layout=dslice_layout)
dslice_slider = FloatRangeSlider(
    value=[np.min(dslice), np.max(dslice)],
    min=np.min(dslice),
    max=np.max(dslice),
    step=0.1,
    description='dp :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
dslice_slider.layout=Layout(width='99%')


In [ ]:
cep = dp_to_cep(dslice)
max_limit = min(np.max(cep), cep_max)
min_limit = np.min(cep)
cep_traces = {
    'type': 'heatmapgl',
    #'type': 'heatmap',
    'z':cep,
    'colorscale': 'Viridis',
    'showscale' : True,
    'hoverinfo' : 'none'
}
cep_layout = {
  'margin': {
    't': 10,
    'r': 10,
    'b': 10,
    'l': 10
  },
  'xaxis' : {
    'showgrid':False,
    'showline':False,
    'zeroline':False,
    #'autorange':'reversed',
    'showticklabels':False,
  },
  'yaxis' : {
    'showgrid':False,
    'showline':False,
    'zeroline':False,
    'showticklabels':False,
  },   
  'shapes' : [{
      'type': 'circle',
      'xref': 'paper',
      'yref': 'paper',
      'x0' : 0.45,
      'y0' : 0.45,
      'x1' : 0.55,
      'y1' : 0.55,
      'line' : {
          'color': 'red',
          'width': 2,
      }
  }]
}
cep_fig = FigureWidget(data=cep_traces, layout=cep_layout)
cep_slider = FloatRangeSlider(
    value=[min_limit, max_limit],
    min=min_limit,
    max=max_limit,
    step=0.1,
    description='cep :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
cep_slider.layout=Layout(width='99%')


In [ ]:
def im_click_fn(trace, points, state):
    if len(points.point_inds) > 0:
        if len(points.xs) > 0:
            xs = points.xs[0]
            ys = points.ys[0]
            im_fig.update_layout({'shapes':[{'x0':xs, 'x1':xs},{'y0':ys, 'y1':ys}]})
            #Updating DP
            dslice_update = dp_slice(fname, xs, ys)
            dmax_limit_update = min(np.max(dslice_update), cep_max)
            dmin_limit_update = np.min(dslice_update)            
            dslice_slider.min = dmin_limit_update
            dslice_slider.max = dmax_limit_update
            dslice_slider.value = [dmin_limit_update, dmax_limit_update]
            dslice_fig.update_traces(z = dslice_update)
            #Updating CEP
            cep_update = dp_to_cep(dslice_update)
            max_limit_update = min(np.max(cep_update), cep_max)
            min_limit_update = np.min(cep_update)            
            cep_slider.min = min_limit_update
            cep_slider.max = max_limit_update
            cep_slider.value = [min_limit_update, max_limit_update]
            cep_fig.update_traces(z = cep_update)

            
im_fig.data[0].on_click(im_click_fn)

def update_limits(change):
    zmin = change["new"][0]
    zmax = change["new"][1]
    dslice_fig.update_traces(zmin = zmin, zmax = zmax)
dslice_slider.observe(update_limits, names='value')

def cep_update_limits(change):
    zmin = change["new"][0]
    zmax = change["new"][1]
    cep_fig.update_traces(zmin = zmin, zmax = zmax)
cep_slider.observe(cep_update_limits, names='value')


In [ ]:
display( 
    VBox([
        im_fig,
        HBox([
            VBox([dslice_fig, dslice_slider], layout=Layout(width='50%')),
            VBox([cep_fig, cep_slider], layout=Layout(width='50%')),
        ])
    ])
)

In [ ]:
#fname = '09_2DWrinkAu_640kx_770mm_70um_0p64mrad_ss8_50x_50y_100z_216step_x256_y256.raw'
#yrows = 130
#xcols = 128
#dtype = np.float32    
#dsize = np.dtype(dtype).itemsize
#num_pixels = 10
#total_size = num_pixels**2*xcols*yrows
#a = np.array(np.random.random_sample((total_size,))*5000, dtype=dtype)
#a.tofile(fname)